# Natural Language Processing < Sentiment Analysis />

In this module, we will work with both Huggingface transformers and LangChain for the task of sentiment analysis.

## Huggingface

### 1) Install Python libraries

In [ ]:
!pip install --no-cache-dir transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 189.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 304.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 311.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 163.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 299.7 MB/s eta 0:00:00


### 2) Analysis function

Here we define our sentiment analysis function.

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=False)
config = AutoConfig.from_pretrained(MODEL)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)
tokenizer.save_pretrained(MODEL)
model.save_pretrained(MODEL)

def run_sentiment_analysis(text):
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  # Print labels and scores
  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  print(text)
  for i in range(scores.shape[0]):
      l = config.id2label[ranking[i]]
      s = scores[ranking[i]]
      print(f"{i+1}) {l} {np.round(float(s), 4)}")
  print('')

### 3) Run sentiment analysis

In [ ]:
run_sentiment_analysis("I am not sure.")
run_sentiment_analysis("There's a great day ahead of us.")
run_sentiment_analysis("Hmm, the stock market did not do very well last week.")

I am not sure.
1) neutral 0.7032
2) negative 0.2462
3) positive 0.0507

There's a great day ahead of us.
1) positive 0.9257
2) neutral 0.0617
3) negative 0.0126

Hmm, the stock market did not do very well last week.
1) negative 0.7485
2) neutral 0.2038
3) positive 0.0477



## OpenAI GPT-3.5

In this section we will create and use a sentiment analysis function using OpenAI's GPT-3.5. You'll need an OpenAI developer API key.

### 1) Install Python libraries

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


### 2) Analysis function

In [ ]:
import os
from openai import OpenAI
os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_DEVELOPER_API_KEY'
client = OpenAI()


def run_sentiment_analysis_gpt(text):
  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are going to help me determine the sentiment of a piece of text. You will simply respond with 'positive', 'negative', or 'neutral'."},
      {"role": "user", "content": text}
    ]
  )
  print(text)
  print(completion.choices[0].message.content)
  print('')

### 3) Run sentiment analysis

In [ ]:
run_sentiment_analysis_gpt("I am not sure.")
run_sentiment_analysis_gpt("There's a great day ahead of us.")
run_sentiment_analysis_gpt("Hmm, the stock market did not do very well last week.")

I am not sure.
neutral

There's a great day ahead of us.
positive

Hmm, the stock market did not do very well last week.
negative

